In [1]:
## Please replace all path to your path before stating to evaluate

In [ ]:
import os
import autogen
from autogen.coding import LocalCommandLineCodeExecutor
from autogen import AssistantAgent, UserProxyAgent
from IPython.display import Image, display
# import fitz  # PyMuPDF
import json
import base64
import re
import time
import pandas as pd
from tqdm.notebook import tqdm
os.environ["OPENAI_API_KEY"] = 'your-api'

In [2]:
def gpt_tokenize(string: str, encoding) -> int:
    """Returns the number of tokens in a text string."""
    num_tokens = len(encoding.encode(string))
    return num_tokens

def find_jpg_files(directory):
    jpg_files = [file for file in os.listdir(directory) if file.lower().endswith('.jpg') or file.lower().endswith('.png')]
    return jpg_files if jpg_files else None

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


def find_excel_files(directory):
    jpg_files = [file for file in os.listdir(directory) if (file.lower().endswith('xlsx') or file.lower().endswith('xlsb') or file.lower().endswith('xlsm')) and not "answer" in file.lower()]
    return jpg_files if jpg_files else None

def read_excel(file_path):
    # 读取Excel文件中的所有sheet
    xls = pd.ExcelFile(file_path)
    sheets = {}
    for sheet_name in xls.sheet_names:
        sheets[sheet_name] = xls.parse(sheet_name)
    return sheets

def dataframe_to_text(df):
    # 将DataFrame转换为文本
    text = df.to_string(index=False)
    return text

def combine_sheets_text(sheets):
    # 将所有sheet的文本内容组合起来
    combined_text = ""
    for sheet_name, df in sheets.items():
        sheet_text = dataframe_to_text(df)
        combined_text += f"Sheet name: {sheet_name}\n{sheet_text}\n\n"
    return combined_text

def read_txt(path):
    with open(path, "r") as f:
        return f.read()

def truncate_text(text, max_tokens=128000):
    # 计算当前文本的token数
    tokens = text.split()
    if len(tokens) > max_tokens:
        # 截断文本以确保不超过最大token数
        text = ' '.join(tokens[-max_tokens:])
    return text

In [3]:
MODEL_LIMITS = {
    "gpt-3.5-turbo-0125": 16_385,
    "gpt-4-turbo-2024-04-09": 128_000,
    "gpt-4o-2024-05-13": 128_000,
    "gpt-4o-mini-2024-07-18": 128_000
}

# The cost per token for each model input.
MODEL_COST_PER_INPUT = {
    "gpt-3.5-turbo-0125": 0.0000005,
    "gpt-4-turbo-2024-04-09": 0.00001,
    "gpt-4o-2024-05-13": 0.000005,
    "gpt-4o-mini-2024-07-18": 0.00000015
}

# The cost per token for each model output.
MODEL_COST_PER_OUTPUT = {
    "gpt-3.5-turbo-0125": 0.0000015,
    "gpt-4-turbo-2024-04-09": 0.00003,
    "gpt-4o-2024-05-13": 0.000015,
    "gpt-4o-mini-2024-07-18": 0.0000006
}



In [4]:
samples = []
with open("./data.json", "r") as f:
    for line in f:
        samples.append(eval(line.strip()))
len(samples)

43

In [5]:

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={"tags": ["gpt-4"]},  # comment out to get all
)

# create an AssistantAgent named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "cache_seed": 41,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0,  # temperature for sampling
    },  # configuration for autogen's enhanced inference API which is compatible with OpenAI API
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        # the executor to run the generated code
        "executor": LocalCommandLineCodeExecutor(work_dir="coding"),
    },
)
start = time.time()
# the assistant receives a message from the user_proxy, which contains the task description
chat_res = user_proxy.initiate_chat(
    assistant,
    message="""What date is today? Compare the year-to-date gain for META and TESLA.""",
    summary_method="reflection_with_llm",
)
consume = time.time() - start

NameError: name 'config_list' is not defined

In [ ]:
chat_res

In [26]:
model = "gpt-3.5-turbo-0125"
prompt_tokens = chat_res.cost['usage_including_cached_inference'][model]['prompt_tokens']
completion_tokens = chat_res.cost['usage_including_cached_inference'][model]['completion_tokens']
cost = chat_res.cost['usage_including_cached_inference'][model]['cost']
summary = chat_res.summary
history = chat_res.chat_history                

In [ ]:
chat_res.cost['usage_including_cached_inference']

In [11]:
consume

0.039936065673828125

In [5]:
def get_response(text, config_list):
    assistant = autogen.AssistantAgent(
        name="assistant",
        llm_config={
            "cache_seed": 41,  # seed for caching and reproducibility
            "config_list": config_list,  # a list of OpenAI API configurations
            "temperature": 0,  # temperature for sampling
        },  # configuration for autogen's enhanced inference API which is compatible with OpenAI API
    )
    
    # create a UserProxyAgent instance named "user_proxy"
    user_proxy = autogen.UserProxyAgent(
        name="user_proxy",
        human_input_mode="NEVER",
        max_consecutive_auto_reply=10,
        is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
        code_execution_config={
            # the executor to run the generated code
            "executor": LocalCommandLineCodeExecutor(work_dir="coding"),
        },
    )
    # the assistant receives a message from the user_proxy, which contains the task description
    chat_res = user_proxy.initiate_chat(
            assistant,
            message=text,
            summary_method="reflection_with_llm",
        )
    return chat_res
    

In [ ]:
# for id in tqdm(range(31, len(samples))):
# for id in tqdm(range(34, 38)):
# model = "gpt-3.5-turbo-0125-autoagent"
# model = "gpt-4o-2024-05-13"
model = "gpt-4-turbo-2024-04-09"
# model = "gpt-4o-mini-2024-07-18"
config_list = [{"model": model, "api_key": "your-api-key"}]
total_cost = 0
for id in tqdm(range(36, len(samples))):
# for id in tqdm([0]):
    # print(sample)

    sample =samples[id]
    if len(sample["questions"]) > 0:

        image = find_jpg_files(os.path.join("./data", sample["id"]))

        
        excels = find_excel_files(os.path.join("./data", sample["id"]))


        introduction = read_txt(os.path.join("./data", sample["id"], "introduction.txt"))
        questions = []
        for question_name in sample["questions"]:
            questions.append(read_txt(os.path.join("./data", sample["id"], question_name+".txt")))
        
    
        
        text = f"The introduction is detailed as follows. \n {introduction}" 
        if excels:
            text += "\n \n The worksheet can be obtained in the path: "
            for excel in excels:
                text += f" {os.path.join('./data',  sample['id'], excel)}"
    
        if image:
            text += f"\n The image can be obtained in the path: {os.path.join('./data',  sample['id'], image[0])} \n"
        
        question_content = ""    
        # print(workbooks)
        answers = []
        all_mess = []
        for question in tqdm(questions):
            # question_content += question
    
            all_context = text + f"The question is detailed as follows. \n {question} \nPlease answer the question. "
            input_t = all_context
            # input_t = truncate_text(all_context, 2000)
            start = time.time()
            cost = 0
            prompt_tokens = completion_tokens = 0
            try:
                response = get_response(input_t, config_list)

                prompt_tokens = response.cost['usage_including_cached_inference'][model]['prompt_tokens']
                completion_tokens = response.cost['usage_including_cached_inference'][model]['completion_tokens']
                cost = response.cost['usage_including_cached_inference'][model]['cost']
                summary = response.summary
                history = response.chat_history
            except Exception as e:
                print(e)
                time.sleep(3)
                
                # cost = 0
                history = "I cannot solve this task."
                summary = "I cannot solve this task."
                # all_mess.append("I cannot solve this task.")
            total_cost += cost
            print("Total cost: ", total_cost)
            answers.append({"id": sample["id"], "model": model, "input": prompt_tokens,
                            "output": completion_tokens, "cost": cost, "time": time.time()-start, 'summary': summary, "history": history })
            # break
    save_path = os.path.join("./save_process", model+"-autoagent")
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    with open(os.path.join(save_path, sample['id'] + ".json"), "w") as f:
        for answer in answers:
            json.dump(answer, f)
            f.write("\n")
    # if total_cost > 100:
    #     break
    # break
            # the assistant receives a message from the user_proxy, which contains the task description
    # print(answers)

